In [8]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain.schema.runnable import RunnableParallel, RunnableBranch, RunnableLambda
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import Literal
from dotenv import load_dotenv
import os

load_dotenv()

repo_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"

llm1 = HuggingFaceEndpoint(
    repo_id = repo_id,
    task = "text-generation", 
    huggingfacehub_api_token=os.getenv("HUGGINGFACEHUB_ACCESS_TOKEN")
)

model = ChatHuggingFace(llm=llm1)



parser1 = StrOutputParser()


class Feedback(BaseModel):

    sentiment : Literal["positive", "negative"] = Field(description="give the sentiment of the feedback")

parser2 = PydanticOutputParser(pydantic_object=Feedback)


prompt1 = PromptTemplate(
    template="classity the sentiment of the following feedback text into positive or negative \n {feedback} \n {format_instruction}",
    input_variables=["feedback"],
    partial_variables={"format_instruction": parser2.get_format_instructions()}
)


classifier_chain = prompt1 | model | parser2

prompt2 = PromptTemplate(
    template="write an appropriate response to this positive feedback \n {feedback}",
    input_variables=["Feedback"]
)

prompt3 = PromptTemplate(
    template="write an appropriate response to this negative feedback \n {feedback}",
    input_variables=["Feedback"]
)

branch_chain = RunnableBranch(
    (lambda x : x.sentiment == "positive", prompt2 | model | parser1),
    (lambda x : x.sentiment == "negative", prompt3 | model | parser1),
    RunnableLambda(lambda x : "could not find sentiment")
)


chain = classifier_chain | branch_chain

result = chain.invoke({"feedback" : "This product was terrible! It broke after only one use and customer service was unhelpful."})

print(result)


 Dear Customer,

We are sorry to hear that you had a negative experience. Your satisfaction is our top priority, and we want to ensure that we are meeting and exceeding your expectations. We appreciate your feedback and would like to make things right. Please contact us so we can discuss your concerns and find a solution.

Thank you for bringing this to our attention.

Best Regards,
[Your Company Name]


In [9]:
chain.get_graph().print_ascii()

    +-------------+      
    | PromptInput |      
    +-------------+      
            *            
            *            
            *            
   +----------------+    
   | PromptTemplate |    
   +----------------+    
            *            
            *            
            *            
  +-----------------+    
  | ChatHuggingFace |    
  +-----------------+    
            *            
            *            
            *            
+----------------------+ 
| PydanticOutputParser | 
+----------------------+ 
            *            
            *            
            *            
       +--------+        
       | Branch |        
       +--------+        
            *            
            *            
            *            
    +--------------+     
    | BranchOutput |     
    +--------------+     
